<a href="https://colab.research.google.com/github/gabelim/DL_Image_classification_explore/blob/main/Dog_Frog_Image_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

import os
# from google.colab import drive
# drive.mount('/content')


`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


Create data set of dogs and frogs with bing-image-downloader

In [2]:
!pip install bing-image-downloader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from bing_image_downloader import downloader

In [ ]:
downloader.download('frogs', limit=50, output_dir='images/train')

In [ ]:
downloader.download('dog', limit=50, output_dir='images/train')

Make test directory and transfer 10 images into it from train.

In [ ]:
# !mkdir /images/test/dog
# !mkdir /images/test/frogs


# !pwd
!mv /content/images/train/dog/Image_{41..50}* /content/images/test/dogs
!mv /content/images/train/frogs/Image_{41..50}* /content/images/test/frogs

Sequential model. Other model is Functional

In [39]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout,Activation
import numpy as np
import matplotlib.pyplot as plt
#CNN with 3 convo blocks, fully connected layer at the end

model = Sequential()
model.add(Conv2D(32,(3,3),input_shape=(64,64,3))) 
#2D convo layer, 32 filters - size 3x3, input_image 64x64, 3 RGB
#pixel intensity 0-255 populate each element in 64,64,3 
#feature maps are output
model.add(Activation('relu'))
#relu activation function: defines output of neuron give an input
model.add(MaxPooling2D(pool_size =(2,2)))
#pooling layer does dimensionality reduction for each feature map

model.add(Conv2D(32,(3,3))) 
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size =(2,2)))
model.add(Conv2D(32,(3,3))) 
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size =(2,2)))

#dropout layer:prevents overfitting
model.add(Flatten()) #flatten feature map to 1D
model.add(Dense(64))# initialize fully connected network
model.add(Activation('relu')) 
model.add(Dropout(0.5)) # dropout layer
model.add(Dense(1)) #initialize another fully connected layer and output n-dimensional vector to the number of classes
model.add(Activation('sigmoid'))

model.compile(optimizer ='rmsprop',   #perform gradient descent
                   loss ='binary_crossentropy', #best loss function for binary classification
                   metrics =['accuracy'])

In [50]:
#data augmentation - flip tilt zoom image : prevent overfitting

from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale =1./255,
                                   shear_range =0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip =True)
test_datagen = ImageDataGenerator(rescale = 1./255)

#set train and test data locations using flow_from_directory
training_set = train_datagen.flow_from_directory('images/train',
                                                target_size=(64,64),
                                                batch_size= 32,
                                                class_mode='binary')

test_set = test_datagen.flow_from_directory('images/test',
                                           target_size = (64,64),
                                           batch_size = 32,
                                           class_mode ='binary')

Found 80 images belonging to 3 classes.
Found 20 images belonging to 3 classes.


In [51]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 activation_5 (Activation)   (None, 62, 62, 32)        0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 activation_6 (Activation)   (None, 29, 29, 32)        0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                  

In [52]:
from IPython.display import display
from PIL import Image
model.fit(training_set,
                        epochs = 5,
                        validation_data =test_set)

Epoch 1/5
3/3 [==============================] - 3s 923ms/step - loss: -3336.6128 - accuracy: 0.5000 - val_loss: -5166.8975 - val_accuracy: 0.5000
Epoch 2/5
3/3 [==============================] - 2s 777ms/step - loss: -6041.4819 - accuracy: 0.5000 - val_loss: -9074.2324 - val_accuracy: 0.5000
Epoch 3/5
3/3 [==============================] - 2s 1s/step - loss: -11005.3252 - accuracy: 0.5000 - val_loss: -14984.7285 - val_accuracy: 0.5000
Epoch 4/5
3/3 [==============================] - 2s 845ms/step - loss: -18070.0430 - accuracy: 0.5000 - val_loss: -23688.3398 - val_accuracy: 0.5000
Epoch 5/5
3/3 [==============================] - 2s 913ms/step - loss: -29615.4492 - accuracy: 0.5000 - val_loss: -34700.4258 - val_accuracy: 0.5000


In [53]:
test_loss, test_acc = model.evaluate(test_set, verbose=2)
print(test_acc)

1/1 - 0s - loss: -3.4700e+04 - accuracy: 0.5000 - 475ms/epoch - 475ms/step
0.5


In [45]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image =image.load_img('images/test/dogs/Image_49.jpg',target_size =(64,64))

test_image =image.img_to_array(test_image)
test_image =np.expand_dims(test_image, axis =0)
result = model.predict(test_image)
if result[0][0] >= 0.5:
    prediction = 'dog'
else:
    prediction = 'frog'
print(prediction)